# The Requests Library

Now that we know how to use BeautifulSoup to get data from HTML files, let's see how we can scrape data from a real website. Unfortunately, Beautifulsoup can't access websites directly. Therefore, in order to access websites, we will use Python's `requests` library. The `requests` library allows us to send web requests and get a website's HTML data. Once the `requests` library gets us the HTML data, we can use Beautifulsoup, just as we did before, to extract the data we want. So let's see an example.

In the code below we will use the `requests` library and BeautifulSoup to get Tesla's `production and sales by quarter` data from a `html table` the following Wikipedia [webpage](https://en.wikipedia.org/wiki/Tesla,_Inc.). This table corresponds to Tesla's production and sales figures since Q1 2013. We will start by importing the `requests` library by using:

```python
import requests
```

We will then use the `requests.get(website)` function to get the source code from our `wikipage`. The `requests.get()` function returns a `Response` object that we will save in the variable `r`. We can get the HTML data we need from this object by using the `.text` method, as shown below. Finally, we'll convert and display the extracted html table into Pandas dataframe.

In [1]:
from bs4 import BeautifulSoup
import requests

import pandas as pd
import numpy as np

In [22]:
# Create a Response object
r = requests.get('https://en.wikipedia.org/wiki/History_of_Tesla,_Inc.#Timeline_of_production_and_sales')

# Get HTML data
html_data = r.text

The `.text` method returns a string, therefore, `html_data` is a string containing the HTML data from our website. Notice, that since `html_data` is a string it can be passed to the BeautifulSoup constructor, and we will do this next, but for now, let's print the `html_data` string to see what it looks like:

In [23]:
# Print the HTML data
print(html_data)

<!DOCTYPE html>
<html class="client-nojs" lang="en" dir="ltr">
<head>
<meta charset="UTF-8"/>
<title>History of Tesla, Inc. - Wikipedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"4b6e404c-f9d3-4c97-8ee0-de506b50220a","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"History_of_Tesla,_Inc.","wgTitle":"History of Tesla, Inc.","wgCurRevisionId":1015669790,"wgRevisionId":1015669790,"wgArticleId":51237650,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Pages with non-numeric formatnum arguments","CS1 German-language sources (de)","CS1 French-language sources (fr)","CS1 Danish-language sour

As we can see, `html_data` indeed contains the HTML data of our website. Notice, that since we are dealing with a real website this time, the HTML file is very long. 

Now that we have the HTML data from our website, we are ready to use BeautifulSoup just as we did before. The only difference is that this time, instead of passing an open filehandle to the BeautifulSoup constructor, we will pass the `html_data` string. So let's pass `html_data` to the BeautifulSoup constructor to get a BeautifulSoup object:

In [24]:
# Create a BeautifulSoup Object
page_content = BeautifulSoup(html_data, 'html.parser')

# Print the BeautifulSoup Object
print(page_content.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   History of Tesla, Inc. - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"4b6e404c-f9d3-4c97-8ee0-de506b50220a","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"History_of_Tesla,_Inc.","wgTitle":"History of Tesla, Inc.","wgCurRevisionId":1015669790,"wgRevisionId":1015669790,"wgArticleId":51237650,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Pages with non-numeric formatnum arguments","CS1 German-language sources (de)","CS1 French-language sources (fr)","CS1 Da

Now that we have a BeautifulSoup object, we can get our sales and production data. To do this, we need to know which tags contain our table data. In order to figure this out, we need to inspect our webpage using our web browser. For this example we will use the Chrome web browser but all web browsers have the same kind of functionality. We begin by going to our wikipage: https://en.wikipedia.org/wiki/Tesla,_Inc.

Next, we hover our mouse over around the [target html table](https://en.wikipedia.org/wiki/Tesla,_Inc.#Production_and_sales_by_quarter). As an example, we will hover over the `table header` row. Next, we right-click on the header title for `Quarter` and a dropdown menu will appear. From this menu we will choose **Inspect**. Once we click on **Inspect** we will see the HTML source code of our webpage appear on the right, as shown in the figure below:

<br>
<figure>
  <img src = "./wikitable.png" width = "100%" style = "border: thin silver solid; padding: 10px">
</figure> 
<br>

If we inspect the html source code on the right panel, we will see that the table header and rows are all contained within the same `<table>` tag. Therefore, we can use the above information to extract the column title from table header and detail data in the rows. In the code below we use BeautifulSoup's `find()` method to find the target `<table>` tag that has `class="wikitable"`.

<br>
<figure>
  <img src = "./table-tag.png" width = "50%" style = "border: thin silver solid; padding: 10px">
</figure> 
<br>

A complete `html table` can be structured in the following way:
```
<table>
  <thead>
    <tr>
      <th>Month</th>
      <th>Sales</th>
    </tr>
  </thead>
  
  <tbody>
    <tr>
      <td>January</td>
      <td>$100</td>
    </tr>
    <tr>
      <td>February</td>
      <td>$80</td>
    </tr>
  </tbody>
  
  <tfoot>
    <tr>
      <td>Sum</td>
      <td>$180</td>
    </tr>
  </tfoot>
</table>
```

But notice that the html table in our wikipage does not have `<thead>` and `<tfoot>` tags. Let's create a BeautifulSoup's object and name it as `wikitable`.

In [25]:
wikitable = page_content.find('table', {'class': 'wikitable'})
wikitable

<table class="wikitable sortable">
<tbody><tr>
<th data-sort-type="date">Date
</th>
<th>Announced
</th>
<th>Company
</th>
<th>Business
</th>
<th data-sort-type="number">Value (USD)
</th>
<th class="unsortable">References
</th></tr>
<tr>
<td align="center">May 8, 2015
</td>
<td>Yes
</td>
<td>Riviera Tool LLC
</td>
<td>Stamping die systems used to form sheet metal parts
</td>
<td align="center">Unknown
</td>
<td align="center"><sup class="reference" id="cite_ref-90"><a href="#cite_note-90">[90]</a></sup>
</td></tr>
<tr>
<td align="center">Nov 8, 2016
</td>
<td>Yes
</td>
<td><a href="/wiki/Tesla_Grohmann_Automation" title="Tesla Grohmann Automation">Grohmann Engineering</a>
</td>
<td>Engineering automation systems
</td>
<td align="center">135 million
</td>
<td align="center"><sup class="reference" id="cite_ref-91"><a href="#cite_note-91">[91]</a></sup>
</td></tr>
<tr>
<td align="center">Nov 21, 2016
</td>
<td>Yes
</td>
<td><a href="/wiki/SolarCity" title="SolarCity">SolarCity</a>
</td>
<t

In [32]:
wikitable = page_content.find_all('table', class_='wikitable')[1]
wikitable

<table class="wikitable">
<tbody><tr style="text-align:center;">
<th>Quarter</th>
<th>Cumulative<br/>production</th>
<th>Total<br/>production</th>
<th>Model S<br/>sales</th>
<th>Model X<br/>sales</th>
<th>Model 3<br/>sales
</th>
<th>Model Y<br/>sales<sup class="reference" id="cite_ref-187"><a href="#cite_note-187">[a]</a></sup></th>
<th>Total<br/>sales<sup class="reference" id="cite_ref-188"><a href="#cite_note-188">[b]</a></sup></th>
<th>In transit<sup class="reference" id="cite_ref-189"><a href="#cite_note-189">[c]</a></sup></th>
<th>Source
</th></tr>
<tr style="text-align:center;">
<td>Q3 2012</td>
<td>N/A</td>
<td>350</td>
<td>250+</td>
<td style="background:#f1f5fa;"></td>
<td style="background:#f1f5fa;"></td>
<td style="background:#f1f5fa;"></td>
<td>250+</td>
<td></td>
<td><sup class="reference" id="cite_ref-190"><a href="#cite_note-190">[187]</a></sup>
</td></tr>
<tr style="text-align:center;">
<td>Q4 2012</td>
<td>N/A</td>
<td>2,750+</td>
<td>2,400</td>
<td style="background:#

Now that we have extracted the `<table>` tag that have `class="wikitable"` as an object, we can use BeautifulSoup's built-in functions to extract the human-readable text inside this table. Let's grab the information inside `<tbody>` tag.

In [33]:
wikitable.tbody

<tbody><tr style="text-align:center;">
<th>Quarter</th>
<th>Cumulative<br/>production</th>
<th>Total<br/>production</th>
<th>Model S<br/>sales</th>
<th>Model X<br/>sales</th>
<th>Model 3<br/>sales
</th>
<th>Model Y<br/>sales<sup class="reference" id="cite_ref-187"><a href="#cite_note-187">[a]</a></sup></th>
<th>Total<br/>sales<sup class="reference" id="cite_ref-188"><a href="#cite_note-188">[b]</a></sup></th>
<th>In transit<sup class="reference" id="cite_ref-189"><a href="#cite_note-189">[c]</a></sup></th>
<th>Source
</th></tr>
<tr style="text-align:center;">
<td>Q3 2012</td>
<td>N/A</td>
<td>350</td>
<td>250+</td>
<td style="background:#f1f5fa;"></td>
<td style="background:#f1f5fa;"></td>
<td style="background:#f1f5fa;"></td>
<td>250+</td>
<td></td>
<td><sup class="reference" id="cite_ref-190"><a href="#cite_note-190">[187]</a></sup>
</td></tr>
<tr style="text-align:center;">
<td>Q4 2012</td>
<td>N/A</td>
<td>2,750+</td>
<td>2,400</td>
<td style="background:#f1f5fa;"></td>
<td style="

In the earlier cell, we use `find()` function to get the html table. Now, we are going to use `findAll()` function to grab all the defined tags inside the BeautifulSoup object. In this case, we want to extract every row in the `<tbody>` tag and we can do so by using `findAll()` function.

In [34]:
wikitable.tbody.findAll('tr')

[<tr style="text-align:center;">
 <th>Quarter</th>
 <th>Cumulative<br/>production</th>
 <th>Total<br/>production</th>
 <th>Model S<br/>sales</th>
 <th>Model X<br/>sales</th>
 <th>Model 3<br/>sales
 </th>
 <th>Model Y<br/>sales<sup class="reference" id="cite_ref-187"><a href="#cite_note-187">[a]</a></sup></th>
 <th>Total<br/>sales<sup class="reference" id="cite_ref-188"><a href="#cite_note-188">[b]</a></sup></th>
 <th>In transit<sup class="reference" id="cite_ref-189"><a href="#cite_note-189">[c]</a></sup></th>
 <th>Source
 </th></tr>, <tr style="text-align:center;">
 <td>Q3 2012</td>
 <td>N/A</td>
 <td>350</td>
 <td>250+</td>
 <td style="background:#f1f5fa;"></td>
 <td style="background:#f1f5fa;"></td>
 <td style="background:#f1f5fa;"></td>
 <td>250+</td>
 <td></td>
 <td><sup class="reference" id="cite_ref-190"><a href="#cite_note-190">[187]</a></sup>
 </td></tr>, <tr style="text-align:center;">
 <td>Q4 2012</td>
 <td>N/A</td>
 <td>2,750+</td>
 <td>2,400</td>
 <td style="background:#f1

In [35]:
wikitable.tbody.find_all('tr')

[<tr style="text-align:center;">
 <th>Quarter</th>
 <th>Cumulative<br/>production</th>
 <th>Total<br/>production</th>
 <th>Model S<br/>sales</th>
 <th>Model X<br/>sales</th>
 <th>Model 3<br/>sales
 </th>
 <th>Model Y<br/>sales<sup class="reference" id="cite_ref-187"><a href="#cite_note-187">[a]</a></sup></th>
 <th>Total<br/>sales<sup class="reference" id="cite_ref-188"><a href="#cite_note-188">[b]</a></sup></th>
 <th>In transit<sup class="reference" id="cite_ref-189"><a href="#cite_note-189">[c]</a></sup></th>
 <th>Source
 </th></tr>, <tr style="text-align:center;">
 <td>Q3 2012</td>
 <td>N/A</td>
 <td>350</td>
 <td>250+</td>
 <td style="background:#f1f5fa;"></td>
 <td style="background:#f1f5fa;"></td>
 <td style="background:#f1f5fa;"></td>
 <td>250+</td>
 <td></td>
 <td><sup class="reference" id="cite_ref-190"><a href="#cite_note-190">[187]</a></sup>
 </td></tr>, <tr style="text-align:center;">
 <td>Q4 2012</td>
 <td>N/A</td>
 <td>2,750+</td>
 <td>2,400</td>
 <td style="background:#f1

As we can see from the html source code above, the first row consists of the column name and followed by the actual data in the subsequent rows. Let's grab the `column names` from the first row and all the `<th>` tags inside the row.

Notice the row index of `[0]` after `findAll('tr')` to get the first row. Then, we can chain the second `findAll()` function to get all the `<th>` tags inside this row.

In [36]:
wikicolumns = wikitable.tbody.findAll('tr')[0].findAll('th')
wikicolumns

[<th>Quarter</th>,
 <th>Cumulative<br/>production</th>,
 <th>Total<br/>production</th>,
 <th>Model S<br/>sales</th>,
 <th>Model X<br/>sales</th>,
 <th>Model 3<br/>sales
 </th>,
 <th>Model Y<br/>sales<sup class="reference" id="cite_ref-187"><a href="#cite_note-187">[a]</a></sup></th>,
 <th>Total<br/>sales<sup class="reference" id="cite_ref-188"><a href="#cite_note-188">[b]</a></sup></th>,
 <th>In transit<sup class="reference" id="cite_ref-189"><a href="#cite_note-189">[c]</a></sup></th>,
 <th>Source
 </th>]

Let's store the column names in a Python object, called `df_columns`, so we can use it to build Pandas dataframe.

In [42]:
df_columns = []

for column in wikicolumns:
    # remove <br/> inside <th> text, such as `<th>Total<br/>production</th>`
    text = column.get_text(strip=True, separator=" ")
    # append the text into df_columns
    df_columns.append(text)

print(np.array(df_columns))

['Quarter' 'Cumulative production' 'Total production' 'Model S sales'
 'Model X sales' 'Model 3 sales' 'Model Y sales [a]' 'Total sales [b]'
 'In transit [c]' 'Source']


Now that we hava stored the column names, now we want to iterate the remaining rows, consisting the real data in this table. We can use Python `for loop` function to iterate from the second row onward. To do so, we need to set the starting index as follows: `wikitable.tbody.findAll('tr')[1:]`. Let's store our dataset in Python object, called `data`.

In [43]:
df_data = []

for row in wikitable.tbody.findAll('tr')[1:]:
    row_data = []
    for td in row.findAll('td'):
        text = td.get_text(strip=True, separator=" ")
        row_data.append(text)
    df_data.append(np.array(row_data))

# print the first 10 data rows
print(df_data[:10])

[array(['Q3 2012', 'N/A', '350', '250+', '', '', '', '250+', '', '[187]'], 
      dtype='<U7'), array(['Q4 2012', 'N/A', '2,750+', '2,400', '', '', '', '2,400', '',
       '[188]'], 
      dtype='<U7'), array(['Q1 2013', 'N/A', '5,000+', '4,900', '', '', '', '4,900', '',
       '[189]'], 
      dtype='<U7'), array(['Q2 2013', 'N/A', 'N/A', '5,150', '', '', '', '5,150', '', '[190]'], 
      dtype='<U7'), array(['Q3 2013', 'N/A', 'N/A', '5,500+', '', '', '', '5,500+', '', '[191]'], 
      dtype='<U7'), array(['Q4 2013', '~34,851', '6,587', '6,892', '', '', '', '6,892', '',
       '[192]'], 
      dtype='<U7'), array(['Q1 2014', '~41,438', '7,535', '6,457', '', '', '', '6,457', '',
       '[193]'], 
      dtype='<U7'), array(['Q2 2014', '~48,973', '8,763', '7,579', '', '', '', '7,579', '',
       '[194]'], 
      dtype='<U7'), array(['Q3 2014', '~57,736', '~7,075', '7,785', '', '', '', '7,785', '',
       '[195]'], 
      dtype='<U7'), array(['Q4 2014', '64,811', '11,627', '9,834', '', ''

Great! We have grabbed the column names and data. But we want to present the data in human-readable structure. We can use [Pandas DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html) library. Pandas DataFrame is similar to Excel spreadsheet and Google Sheet. This library provides a convenient data structure to manipulate and present data with Python.

Let's create a Panda DataFrame object, called `dataframe`, so we can present our data in a spreadsheet structure.

In [44]:
dataframe = pd.DataFrame(data=df_data, columns=df_columns)
dataframe.set_index('Quarter', inplace=True)
dataframe

,Cumulative production,Total production,Model S sales,Model X sales,Model 3 sales,Model Y sales [a],Total sales [b],In transit [c],Source
Quarter,,,,,,,,,
Q3 2012,N/A,350,250+,,,,250+,,[187]
Q4 2012,N/A,"2,750+","2,400",,,,"2,400",,[188]
Q1 2013,N/A,"5,000+","4,900",,,,"4,900",,[189]
Q2 2013,N/A,N/A,"5,150",,,,"5,150",,[190]
Q3 2013,N/A,N/A,"5,500+",,,,"5,500+",,[191]
Q4 2013,"~34,851","6,587","6,892",,,,"6,892",,[192]
Q1 2014,"~41,438","7,535","6,457",,,,"6,457",,[193]
Q2 2014,"~48,973","8,763","7,579",,,,"7,579",,[194]
Q3 2014,"~57,736","~7,075","7,785",,,,"7,785",,[195]


Great job! We have extracted Tesla's production and sales data from an `html table` in a Wikipage and converted the data into Python and Pandas DataFrame. It's now your time to practice with `requests` and `BeautifulSoup` libraries.

# TODO: Get Amazon financial data from Wikipage

URL links:
- Wikipage: https://en.wikipedia.org/wiki/Amazon_(company) <br/>
- Financial data: https://en.wikipedia.org/wiki/Amazon_(company)#Finances

Tasks: <br/>
Start by importing the `BeautifulSoup` and `requests` libraries. Then use the `requests.get()` function with the appropriate `params` to get our website's HTML data. Then create a BeautifulSoup Object named `page_content` using our website's HTML data and the `html_parser` parser. Then use the `find()` method to find the `<table>` tag. Then, get the table column names. Finally, create a loop that prints all the countries and population from `<tbody>` tag.

In [46]:
# Import libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

# Create a Response object
r = requests.get('https://en.wikipedia.org/wiki/Amazon_(company)')

# Get HTML data
html_data = r.text

# Create a BeautifulSoup Object
page_content = BeautifulSoup(html_data, 'html.parser')

# Find financial table
wikitable = page_content.find('table', {'class':'wikitable float-left'})

# Find all column titles
wikicolumns = wikitable.find_all('tr')[0].find_all('th')

# Loop through column titles and store into Python array
df_columns = []

for column in wikicolumns:
    text = column.get_text(strip=True, separator=" ")
    df_columns.append(text)


# Loop through the data rows and store into Python array
df_data = []

for row in wikitable.find_all('tr')[1:]:
    row_data = []
    for td in row.find_all('td'):
        text = td.get_text(strip=True, separator=" ")
        row_data.append(text)
    df_data.append(np.array(row_data))

# Print financial data in DataFrame format and set `Year` as index
dataframe = pd.DataFrame(data=df_data, columns=df_columns)
dataframe.set_index('Year', inplace=True)
dataframe

,Revenue in mil. USD$,Net income in mil. USD$,Total Assets in mil. USD$,Employees
Year,,,,
2007 [138],"14,835",476,"6,485","17,000"
2008 [139],"19,166",645,"8,314","20,700"
2009 [140],"24,509",902,"13,813","24,300"
2010 [141],"34,204","1,152","18,797","33,700"
2011 [142],"48,077",631,"25,278","56,200"
2012 [143],"61,093",−39,"32,555","88,400"
2013 [144],"74,452",274,"40,159","117,300"
2014 [145],"88,988",−241,"54,505","154,100"
2015 [146],"107,006",596,"64,747","230,800"


# XML

Throughout these lessons we have used HTML files and BeautifulSoup's `html_parser` to show you how to scrape data. We should note that the exact same techniques can be applied to XML files. The only difference is that you will have to use an XML parser in the BeautifulSoup constructor. For example, in order to parse a document as XML, you can use `lxml`’s XML parser by passing in `xml` as the second argument to the BeautifulSoup constructor:

```python
page_content = BeautifulSoup(xml_file, 'xml')
```

The above statement will parse the given `xml_file` as XML using the `xml` parser.

# Final Remarks

So now you should know how to scrape data from websites using the `requests` and `BeautifulSoup` libraries. We should note, that you should be careful when scrapping websites not to overwhelm a website's server. This can happen if you write computer programs that send out a lot of requests very quickly. Doing this, will overwhelm the server and probably cause it to get stuck. This is obviously very bad, so avoid making tons of web requests in a short amount of time. In fact, some servers monitor if you are making too many requests and block you, if you are doing so. So keep this in mind when you are writing computer programs.

# Solution

[Solution notebook](requests_library_solution.ipynb)